In [2]:
import pandas as pd
#Z:\Asset Management\FUNDOS e CLUBES\Gerencial\dashboard LFT.xlsx
dados = pd.read_excel(r'Z:\Asset Management\FUNDOS e CLUBES\Gerencial\dashboard LFT.xlsx', sheet_name='Historico preços')
#Renomear a coluna 'Unnamed: 0' para 'Data'
dados.rename(columns={'Unnamed: 0': 'Data'}, inplace=True)
#Dropar as primeiras duas linhas
dados.drop(index=[0, 1], inplace=True)
#Ficar somente com a coluna 'BLFT 0 06/01/30'
dados = dados[['Data', 'BLFT 0 06/01/30']]
#Renomear a coluna 'BLFT 0 06/01/30' para 'RetornoLFT'
dados.rename(columns={'BLFT 0 06/01/30': 'RetornoLFT'}, inplace=True)

dados.to_csv('Dados/dados_lft.csv', index=False)
dados

,Data,RetornoLFT
2,2024-07-14 00:00:00,NaN
3,2024-07-15 00:00:00,14927.313594
4,2024-07-16 00:00:00,14933.628116
5,2024-07-17 00:00:00,14940.367855
6,2024-07-18 00:00:00,14947.563505
...,...,...
363,2025-07-10 00:00:00,16812.95587
364,2025-07-11 00:00:00,16821.903561
365,2025-07-12 00:00:00,NaN
366,2025-07-13 00:00:00,NaN


In [ ]:
#Renomear a coluna 'Unnamed: 0' para 'Data'
dados.rename(columns={'Unnamed: 0': 'Data'}, inplace=True)
#Dropar as primeiras duas linhas
dados.drop(index=[0, 1], inplace=True)
#Ficar somente com a coluna 'BLFT 0 06/01/30'
dados = dados[['Data', 'BLFT 0 06/01/30']]
#Renomear a coluna 'BLFT 0 06/01/30' para 'RetornoLFT'
dados.rename(columns={'BLFT 0 06/01/30': 'RetornoLFT'}, inplace=True)

dados.to_csv('dados_lft.csv', index=False)

C:\Users\emanuel.cabral\AppData\Local\Temp\ipykernel_30364\1061845245.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dados.rename(columns={'BLFT 0 06/01/30': 'RetornoLFT'}, inplace=True)


,Data,RetornoLFT
2,2024-07-10 00:00:00,16775.731744
3,2024-07-11 00:00:00,16776.137799
4,2024-07-12 00:00:00,16776.628449
5,2024-07-13 00:00:00,NaN
6,2024-07-14 00:00:00,NaN
...,...,...
363,2025-07-06 00:00:00,NaN
364,2025-07-07 00:00:00,16822.546481
365,2025-07-08 00:00:00,16822.800265
366,2025-07-09 00:00:00,16822.952535


In [4]:
import requests
SGS_ID = 12
url = f"https://api.bcb.gov.br/dados/serie/bcdata.sgs/{SGS_ID}/dados?formato=json"

resp = requests.get(
    url,
    headers={"Accept": "application/json", "User-Agent": "Mozilla/5.0"},
    timeout=10,
)
resp.raise_for_status()

df = (pd.DataFrame(resp.json())              # [{'data':'01/07/2024', 'valor':'0.0155'}, …]
        .rename(columns={"data": "Data", "valor": "cdi"})
        .assign(Data=lambda d: pd.to_datetime(d["Data"], dayfirst=True),
                cdi =lambda d: pd.to_numeric(d["cdi"],
                                                errors="coerce") / 100)  # → decimal
        .dropna(subset=["cdi"]))

df.set_index("Data")["cdi"].sort_index()

HTTPError: 404 Client Error: Not Found for url: https://api.bcb.gov.br/dados/serie/bcdata.sgs/12/dados?formato=json

In [5]:
def load_cdi_series(cache_csv: str = "Dados/cdi_cached.csv") -> pd.Series:
    """
    Retorna Series diária do CDI (decimal) indexada por datetime.

    1) Tenta ler o cache CSV (muito mais rápido).
    2) Se não existir ou estiver vazio, faz download da série 12 do SGS
       via requests e grava o cache.
    """
    # ───────── tenta cache ───────────────────────────────────────
    try:
        s = (pd.read_csv(cache_csv, parse_dates=["Data"])
                .set_index("Data")["cdi"]
                .astype(float)
                .sort_index())
        if not s.empty:
            return s
    except FileNotFoundError:
        pass                                          # segue para o download

    # ───────── download via API do BCB ───────────────────────────
    SGS_ID  = 12                      # CDI Over / Taxa DI
    dt_ini  = "2000-01-01"            # pode ajustar
    dt_fim  = dt.date.today().strftime("%Y-%m-%d")

    url = (f"https://api.bcb.gov.br/dados/serie/bcdata.sgs/{SGS_ID}/dados"
           f"?formato=json&dataInicial={dt_ini}&dataFinal={dt_fim}")

    resp = requests.get(url, timeout=15)
    resp.raise_for_status()           # dispara se 4xx/5xx

    df = (pd.DataFrame(resp.json())          # [{'data':'01/07/2024','valor':'0.0155'}, …]
            .rename(columns={"data": "Data", "valor": "cdi"})
            .assign(Data=lambda d: pd.to_datetime(d["Data"], dayfirst=True),
                    cdi =lambda d: pd.to_numeric(d["cdi"],
                                                 errors="coerce") / 100)   # 1,55% → 0.0155
            .dropna(subset=["cdi"]))

    serie = df.set_index("Data")["cdi"].sort_index()

    # guarda cache para os próximos 24 h
    serie.to_csv(cache_csv, index=True)

    return serie
df = load_cdi_series()
df

Data
2025-01-02    0.000455
2025-01-03    0.000455
2025-01-06    0.000455
2025-01-07    0.000455
2025-01-08    0.000455
                ...   
2025-07-07    0.000551
2025-07-08    0.000551
2025-07-09    0.000551
2025-07-10    0.000551
2025-07-11    0.000551
Name: cdi, Length: 131, dtype: float64

In [ ]:
import